# Lab 1
### Comparison of pattern matching algorithms:
* naive pattern matching
* finite automaton algorithm
* Knuth-Morris-Pratt

In [1]:
#import re
from sys import argv
from time import time

### Naive $O(nm)$

In [2]:
def naive(text, pattern):
    n = len(text)
    m = len(pattern)
    found = []
    for s in range(n - m + 1):  # O(n - m + 1)
        if (text[s : m + s] == pattern):  # O(m)
            found.append(s)

    return found

### Finite automaton $O(m^3A + n)$
where $A$ is the size of alaphabet

In [3]:
def transition_table(pattern):
    alphabet = list(pattern)
    result = []
    for q in range(0, len(pattern) + 1):
        result += [{}]
        for a in alphabet:
            k = min(len(pattern) + 1, q + 2)
            while True:
                k = k - 1
                if (len(pattern[:k]) == 0 or pattern[:k] == pattern[q - k + 1:q] + a):
                    break
            result[q][a] = k
    return result

In [4]:
def fa_string_matching(text, delta):
    q = 0
    found = []
    for s in range(0, len(text)):
        q = delta[q].get(text[s], 0)
        if(q == len(delta) - 1):
            found.append(s - len(delta) + 2)

    return found

### Knuth-Morris-Pratt $O(n+m)$

In [5]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi += [k]
    return pi

In [6]:
def kmp_string_matching(text, prefix_function, pattern):
    found = []
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = prefix_function[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            found += [i + 1 - q]
            q = prefix_function[q-1]

    return found

In [7]:
def measurement(text, pattern, trials = 3, show = False):

    naive_time = 0.0
    for _ in range(trials):
        start = time()
        found_na = naive(text, pattern)
        end = time()
        naive_time += (end - start) / trials
    print(f"Naive time:                     {naive_time : .2f} s")


    transition_func_time = 0.0
    for _ in range(trials):
        start = time()
        transition_func = transition_table(pattern)
        end = time()
        transition_func_time += (end - start) / trials
    print(f"Preprocessing finite automaton: {transition_func_time : .2f} s")

    finite_automaton_time = 0.0
    for _ in range(trials):
        start = time()
        found_fa = fa_string_matching(text, transition_func)   
        end = time()
        finite_automaton_time += (end - start) / trials
    print(f"Finite automaton:               {end - start : .2f} s")

    prefix_func_time = 0.0
    for _ in range(trials):
        start = time()
        prefix_func = prefix_function(pattern)
        end = time()
        prefix_func_time += (end - start) / trials
    print(f"Preprocessing kmp:              {prefix_func_time : .2f} s")

    kmp_time = 0.0
    for _ in range(trials):
        start = time()
        found_kmp = kmp_string_matching(text, prefix_func, pattern)
        end = time()
        kmp_time += (end - start) / trials
    print(f"KMP:                            {end - start : .2f} s")

    print("\nAre results the same?")
    print(f"naive\t=  fa: { 'YES' if (found_na == found_fa) else 'NO' }")
    print(f"naive\t= kmp: { 'YES' if (found_na == found_kmp) else 'NO' }")
    print(f"fa\t= kmp: { 'YES' if (found_fa == found_kmp) else 'NO' }")

    if (show):
        print("\nPattern positions:")
        for s in found_kmp:
            print(s)


### "art" pattern appearances in polish act from 1997 found using all three algorightms and time comparison.

In [8]:
filename = "tests/1997_714.txt"
with open(filename, "r") as f:
    text = f.read()
pattern = "art"

measurement(text, pattern, trials = 3, show=True)

Naive time:                      0.03 s
Preprocessing finite automaton:  0.00 s
Finite automaton:                0.04 s
Preprocessing kmp:               0.00 s
KMP:                             0.03 s

Are results the same?
naive	=  fa: YES
naive	= kmp: YES
fa	= kmp: YES

Pattern positions:
1156
1505
4692
4734
4879
5082
5148
5949
6039
7266
7511
7781
8044
8299
9104
9959
10022
10224
11122
11207
11618
13194
15284
15358
16092
16261
16406
16547
16616
16840
16856
23637
24061
24152
24586
24683
24780
24931
25530
25689
27001
27288
27479
27542
27592
27857
28373
28558
28766
30964
31021
31096
31362
31811
32609
32968
33053
33268
33595
34651
34737
35511
36155
37143
37543
38451
38595
39056
39210
39436
39568
39980
41152
41829
42028
42198
42371
42504
42718
42896
42941
43447
43555
43787
44590
44653
44953
45010
45293
45401
47319
47422
48785
48820
48906
49052
49259
49316
49488
49559
49915
49979
50102
50160
50702
51050
51179
51966
52071
52272
52552
53008
53032
53211
53788
53931
54078
54137
54770
55075
55279

### Search for "kruszwil" pattern in fragment of polish Wikipedia with time comparison.

In [9]:
filename = "tests/wikipedia-tail-kruszwil.txt"
with open(filename, "r") as f:
    text = f.read()
pattern = "kruszwil"

measurement(text, pattern, trials=3, show=True)

Naive time:                      34.60 s
Preprocessing finite automaton:  0.00 s
Finite automaton:                51.57 s
Preprocessing kmp:               0.00 s
KMP:                             38.91 s

Are results the same?
naive	=  fa: YES
naive	= kmp: YES
fa	= kmp: YES

Pattern positions:
42549966
50733405
281449082
281449554
281450440
281450563
281450687
281450990
281451486
281451934
281452175
281453212
281453705


It is worth to notice, that naive algorithm gives best results. It is probably due, to the fact that it is really simple and in natural language there are hardly ever occasions for it to match falsely more than only a tiny fraction of the whole pattern, so the repetitions of checking the text chunks, which make up the pessimistic time, do not occur.

### Pattern for which elapsed time for asymptotically faster algorithms is at least two times shorter than for naive algorithm (without preprocessing). Also the preprocessing for automaton takes much more time than preprocessing for KMP.

In [10]:
multiplier_A = 15000
multiplier_text = 200
text = ("a" * multiplier_A + "b") * multiplier_text
pat = multiplier_A * "a" + "a"
measurement(text, pat, trials=1)

Naive time:                      1.68 s
Preprocessing finite automaton:  252.43 s
Finite automaton:                0.48 s
Preprocessing kmp:               0.00 s
KMP:                             0.76 s

Are results the same?
naive	=  fa: YES
naive	= kmp: YES
fa	= kmp: YES


#### Bibliography
* [1] Knuth, Donald; Morris, James H.; Pratt, Vaughan (1977). "*Fast pattern matching in strings*". SIAM Journal on Computing.
* [2] Thomas H. Cormen, Charles E. Leiserson, Ronald L. Rivest, and Clifford Stein. *Introduction to Algorithms*, Second Edition. MIT Press and McGraw-Hill, 2001.